### P0 Code Generator for WASM is extended with explicit allocation and deallocation

Group-01, March 2020

#### What is modified?
    - genVar
    - genGlobalVars
    - loadItem
    - genSelect
    - genIndex
    - genAssign

#### What is new?

    - genAssign1
    - genNew
    - genDispose
    - genPtr
    - genNewHeap
    - genShowHeap


### P0 Code Generator for WASM
#### Emil Sekerinski, March 2019

The generated code is kept in memory and all code generation procedures continuously append to that code: procedure `genProgStart` initializes the generator, then `gen`-prefixed procedures are to be called for P0 constructs in the same order in which they are recognized by a recursive descent parser, and finally procedure `genProgExit` returns the generated code in assembly language as a string in textual WebAssembly. The generation procedures are:
- `genBool`, `genInt`, `genRec`, `genArray`, `genPtr`
- `genProgStart`, `genGlobalVars`, `genProgEntry`, `genProgExit`
- `genProcStart`, `genFormalParams`, `genLocalVars`, `genProcEntry`, `genProcExit`
- `genSelect`, `genIndex`, `genVar`, `genConst`, `genUnaryOp`, `genBinaryOp`, `genRelation`
- `genAssign`, `genActualPara`, `genCall`, `genRead`, `genWrite`, `genWriteln`
- `genSeq`, `genThen`, `genIfThen`, `genElse`, `genIfElse`, `genWhile`, `genDo`, `genWhileDo`
- `genAssign1`, `genNew`, `genDispose` 

Errors in the code generator are reported by calling `mark` of the scanner. The data types of the symbol table are used to specify the P0 constructs for which code is to be generated.

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from SC import TIMES, DIV, MOD, AND, PLUS, MINUS, OR, EQ, NE, LT, GT, LE, \
     GE, NOT, CARET, mark
from ST import indent, Var, Ref, Const, Type, Proc, StdProc, Int, Bool, Array, Pointer, Record, find
from HM import *
heap = None

Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb
Importing Jupyter notebook from HM.ipynb
Importing Jupyter notebook from P0.ipynb


Following variables determine the state of the code generator:

- `curlev` is the current level of nesting of P0 procedures
- `memmax` is the size of the memory, in which pointers, records and arrays are allocated
- `asm` is a list of strings with the WASM instruction in textual form

Procedure `genProgStart()` initializes these variables.

In [2]:
def genProgStart():
    global curlev, memsize, asm, heapmemsize
    curlev, memsize, heapmemsize = 0, 0, 0
    asm = ['(module',
           '(import "P0lib" "write" (func $write (param i32)))',
           '(import "P0lib" "writeln" (func $writeln))',
           '(import "P0lib" "read" (func $read (result i32)))']

Following procedures "generate code" for all P0 types by determining the size of objects and store in the `size` field.
- Integers and booleans occupy 4 bytes
- The size of a record is the sum of the sizes of its field; the offset of a field is the sum of the size of the preceding fields
- The size of an array is its length times the size of the base type.
- The size of a `pointer` occupy 4 bytes for only storing the adr of the heap allocated object that it points to. The heapsize of a pointer is the size of its base type.

In [3]:
def genBool(b):
    # b is Bool
    b.size = 4; return b

def genInt(i):
    # i is Int
    i.size = 4; return i

def genRec(r):
    # r is Record
    s = 0
    for f in r.fields:
        f.offset, s = s, s + f.tp.size
    r.size = s
    return r

def genArray(a: Array):
    # a is Array
    a.size = a.length * a.base.size
    return a

## new code added here
def genPtr(p: Pointer):
    p.size = 4 # size of the a Pointer on the stack, for storing the adr on the heap
    if p.base == Int or p.base == Bool:
        p.heapsize = 4
    elif type(p.base) == Array:
        p.heapsize = p.base.length * p.base.base.size
    elif type(p.base) == Record:
        s = 0
        for f in p.base.fields:
            f.offset, s = s, s + f.tp.size
        p.heapsize = s
    return p

Global variable is extended with `Pointer` variable.

In [4]:
def genGlobalVars(sc, start):
    global memsize
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            if sc[i].tp in (Int, Bool):
                asm.append('(global $' + sc[i].name + ' (mut i32) i32.const 0)')
            elif type(sc[i].tp) in (Array, Record, Pointer):
                sc[i].lev, sc[i].adr, sc[i].start, memsize = -2, memsize, memsize, memsize + sc[i].tp.size #x.start is used to handle the unique pointer id
                sc[i].index = 0 # for accessing heap variable (important for array or record)
            else: mark('WASM: type?')
    
def genLocalVars(sc, start):
    for i in range(start, len(sc)):
        if type(sc[i]) == Var:
            if sc[i].tp in (Int, Bool):
                asm.append('(local $' + sc[i].name + ' i32)')
            elif type(sc[i].tp) in (Array, Record, Pointer):
                mark('WASM: no local arrays, records, pointers')
            else: mark('WASM: type?')
    return None

Procedure `loadItem(x)` generates code for loading `x` on the expression stack, assuming `x` is global `Var`, local `Var`, stack `Var`, memory `Var`, local `Ref`, stack `Ref`, `Const`.

If `x` has the type of Pointer, it loads the address it points to (and add the address with index if needed) and gets the value at that address.

In [5]:
def loadItem(x):
    if type(x) == Var:
        if x.lev == 0: asm.append('global.get $' + x.name) # global Var
        elif x.lev == curlev: asm.append('local.get $' + x.name) # local Var
        elif x.lev == -2: # memory Var
            if type(x.tp) == Pointer: # pointer var
                asm.append('i32.const ' + str(x.adr))
                asm.append('i32.load') # load the value of the address pointed by a pointer
                if x.index != 0:
                    asm.append('i32.const '+ str(x.index))
                    asm.append('i32.add') # if the pointer points to a non Int or Bool object
                asm.append('i32.load')
            else: # array or record
                asm.append('i32.const ' + str(x.adr))
                asm.append('i32.load')
        elif x.lev != -1: mark('WASM: var level!') # already on stack if lev == -1
    elif type(x) == Ref:
        if x.lev == -1: asm.append('i32.load')
        elif x.lev == curlev:
            asm.append('local.get $' + x.name)
            asm.append('i32.load')
        else: mark('WASM: ref level!')
    elif type(x) == Const: asm.append('i32.const ' + str(x.val))

In [6]:
def genVar(x):
    # x is Var, Ref
    if 0 < x.lev < curlev: mark('WASM: level!')
    if type(x) == Ref:
        y = Ref(x.tp); y.lev, y.name = x.lev, x.name
        # if type(x.tp) in (Array, Record):
        #    if x.lev > 0: y.name = x.name 
    elif type(x) == Var:
        y = Var(x.tp); y.lev, y.name = x.lev, x.name
        # if x.lev >= 0: y.name = x.name
        if x.lev == -2: 
            y.adr = x.adr; y.start = x.start
            y.index = x.index
    return y

In [7]:
def genConst(x):
    # x is Const
    return x

In [8]:
def genUnaryOp(op, x):
    loadItem(x)
    if op == MINUS:
        asm.append('i32.const -1')
        asm.append('i32.mul')
        x = Var(Int); x.lev = -1
    elif op == NOT:
        asm.append('i32.eqz')
        x = Var(Bool); x.lev = -1
    elif op == AND:
        asm.append('if (result i32)')
        x = Var(Bool); x.lev = -1
    elif op == OR:
        asm.append('if (result i32)')
        asm.append('i32.const 1')
        asm.append('else')
        x = Var(Bool); x.lev = -1
    else: mark('WASM: unary operator?')
    return x

In [9]:
def genBinaryOp(op, x, y):
    if op in (PLUS, MINUS, TIMES, DIV, MOD):
        loadItem(x); loadItem(y)
        asm.append('i32.add' if op == PLUS else \
                   'i32.sub' if op == MINUS else \
                   'i32.mul' if op == TIMES else \
                   'i32.div_s' if op == DIV else \
                   'i32.rem_s' if op == MOD else '?')
        x = Var(Int); x.lev = -1
    elif op == AND:
        loadItem(y) # x is already on the stack
        asm.append('else')
        asm.append('i32.const 0')
        asm.append('end')
        x = Var(Bool); x.lev = -1
    elif op == OR:
        loadItem(y) # x is already on the stack
        asm.append('end')
        x = Var(Bool); x.lev = -1
    else: assert False
    return x

In [10]:
def genRelation(op, x, y):
    loadItem(x); loadItem(y)
    asm.append('i32.eq' if op == EQ else \
               'i32.ne' if op == NE else \
               'i32.lt_s' if op ==  LT else \
               'i32.gt_s' if op == GT else \
               'i32.le_s' if op == LE else \
               'i32.ge_s' if op == GE else '?')
    x = Var(Bool); x.lev = -1
    return x

Procedure `genSelect(x, f)` generates code for `x.f`, provided `f` is in `x.fields`. If `x` is `Var`, i.e. allocated in memory, only `x.adr` is updated and no code is generated. If `x` is `Ref`, i.e. a reference to memory, code for adding the offset of `f` is generated. An updated item is returned.

Procedure `genSelect(x, f)` is extended for Pointer Var. If x is a Pointer Var and f is a field of the record in this  Pointer, `x.index` is updated. `x.select` is updated based on the type of selected field.

In [11]:
def genSelect(x, f):
    # x.f, assuming x.tp is Record or x.tp is Pointer where x.tp.base is Record or x.select is Record
    # x is global Var, local Ref, stack Ref
    # and f is Field
    if type(x) == Var:
        if type(x.tp) == Pointer: #modified
            if type(x.tp.base) == Record: # x.f^
                for j in x.tp.base.fields:
                    if j.name == f.name:
                        x.index += f.offset
                        x.select = f.tp # x.select becomes the type of the selected field, f
                        return x
                else:
                    x.select = x.tp.base
                    mark("error: "+str(f.name)+" is not a field")
                    return x
            else: # x[1].f^ or x.f.f^
                for j in x.select.fields:
                    if j.name == f.name:
                        x.index += f.offset
                        x.select = f.tp
                        return x
                else:
                    x.select = x.tp.base
                    mark("error: "+str(f.name)+" is not a field")
                    return x
        else:
            x.adr += f.offset
            x.select = f.tp
    elif type(x) == Ref:
        if x.lev > 0: asm.append('local.get $' + x.name)
        asm.append('i32.const ' + str(f.offset))
        asm.append('i32.add')
        x.lev = -1
    x.tp = f.tp
    #####
    if type(x.tp) == Pointer and type(x.tp.base) == Array:
        x.dim = 1 #dimension of the array
    #####
    return x

Procedure `genIndex(x, y)` generates code for `x[y]`, assuming `x` is `Var` or `Ref`, `x.tp` is `Array`, and `y.tp` is `Int`. If `y` is `Const`, only `x.adr` is updated and no code is generated, otherwise code for array index calculation is generated.

`genIndex(x, y)` is extended for Pointer Var. In this case, `x.tp.base` is Array or `x.select` is Array. `x.adr` and `x.index` is updated if needed and no code is generated. `y.tp` can only be `Int` at this point.

In [12]:
def genIndex(x, y):
    # x[y], assuming x.tp is Array or x.tp is Pointer with base type or selected type of Array
    # x is global Var, local Ref, stack Ref
    # and y is Const, local Var, global Var, stack Var
    if type(x) == Var: # at x.adr
        if type(x.tp) == Pointer: # extended
            if type(y) == Const:
                if type(x.tp.base) in {Record}: # x.select has to be Array
                    x.index += (y.val - x.select.lower) * x.select.base.size # index of the Node.value
                    x.select = x.select.base
                elif type(x.tp.base) in {Array}:
                    if x.dim == 1: # x.tp.base is 1 dim array
                        x.index += (y.val - x.tp.base.lower) * x.tp.base.size # index of the Node.value
                        x.select = x.tp.base.base
                        x.dim += 1
                    else: # x.tp.base is n dim array
                        # x[e][e]^
                        x.index += (y.val - x.select.lower) * x.select.base.size # index of the Node.value
                        x.select = x.select.base
                        x.dim += 1
            else:
                mark("error: should not be int at this stage. ") # can be extended in further steps
        elif type(y) == Const: #x is an array var
            x.adr += (y.val - x.tp.lower)*x.tp.base.size
            x.tp = x.tp.base
        else: # y is global Var, local Var, stack Var
            loadItem(y) # y on stack
            if x.tp.lower != 0:
                asm.append('i32.const ' + str(x.tp.lower))
                asm.append('i32.sub')
            asm.append('i32.const ' + str(x.tp.base.size))
            asm.append('i32.mul')
            asm.append('i32.const ' + str(x.adr))
            asm.append('i32.add')
            x = Ref(x.tp.base); x.lev = -1
    else: # x is local Ref, stack Ref; y is Const, global Var, local Var, stack Var
        if x.lev == curlev: loadItem(x); x.lev = -1
        if type(y) == Const:
            asm.append('i32.const ' + str((y.val - x.tp.lower) * x.tp.base.size))
            asm.append('i32.add')
        else:
            loadItem(y) # y on stack
            asm.append('i32.const ' + str(x.tp.lower))
            asm.append('i32.sub')
            asm.append('i32.const ' + str(x.tp.base.size))
            asm.append('i32.mul')
            asm.append('i32.add')
        x.tp = x.tp.base
    return x

Procedure `genAssign(x, y)` generates code for `x := y`, provided `x` is `Var`, `Ref` and `y` is `Var`, `Ref`.

`genAssign(x,y)` is exntended. Now, the procedure `genAssign(x, y)` will also generateds code for pointer variable.

In [13]:
def genAssign(x, y):
    if type(x) == Var:
        if x.lev == -2: 
            if type(x.tp) != Pointer:
                asm.append('i32.const ' + str(x.adr))
            else: # x.tp is Pointer, then loads the address pointed to the heap allocated object
                asm.append('i32.const ' + str(x.adr))
                asm.append('i32.load') # load the adr pointed by x
                if x.index != 0:
                    asm.append('i32.const ' + str(x.index))
                    asm.append('i32.add')
        loadItem(y)
        if x.lev == 0: asm.append('global.set $' + x.name)
        elif x.lev == curlev: asm.append('local.set $' + x.name)
        elif x.lev == -2: 
            asm.append('i32.store')
        else: mark('WASM: level!')
    elif type(x) == Ref:
        if x.lev == curlev: asm.append('local.get $' + x.name)
        loadItem(y)
        asm.append('i32.store')
    else: assert False

Procedure `genAssign1(x,y)` generates code for `x := y`, provided `x` and `y` are `Var` with Pointer type. `x.adr` is the adr of Pointer x in the WebAssembly Memory and `y.adr` is the adr of Pointer y in the WebAssembly Memory. Store the address pointed by y to x.

In [ ]:
def genAssign1(x, y):
    asm.append('i32.const ' + str(x.adr))
    asm.append('i32.const ' + str(y.adr))
    asm.append('i32.load')
    asm.append('i32.store')

Procedure `genNew(x)` generates code for explicit allocation, provided x is a Pointer Var.

Procedure `genDispose(x)` generates code for explicit deallocation, provided x is a Pointer Var.

In [14]:
def genProgEntry(ident):
    asm.append('(func $program')

def genProgExit(x):
    asm.append(')\n(memory ' + str(memsize // 2** 16 + 1) + ')\n(start $program)\n)')
    return '\n'.join(l for l in asm)

def genProcStart(ident, fp):
    global curlev
    if curlev > 0: mark('WASM: no nested procedures')
    curlev = curlev + 1
    asm.append('(func $' + ident + ' ' + ' '.join('(param $' + e.name + ' i32)' for e in fp) + '')
    for p in fp:
        if p.tp in (Int, Bool) and type(p) == Ref:
            mark('WASM: only array and record reference parameters')
        elif type(p.tp) in (Array, Record) and type(p) == Var:
            mark('WASM: no structured value parameters')

def genProcEntry(ident, parsize, localsize):
    pass

def genProcExit(x, parsize, localsize):
    global curlev
    curlev = curlev - 1
    asm.append(')')

def genActualPara(ap, fp, n):
    if type(fp) == Ref:  #  reference parameter, assume ap is Var
        if ap.lev == -2: asm.append('i32.const ' + str(ap.adr))
        # else ap.lev == -1, on stack already
    elif type(ap) in (Var, Ref, Const): loadItem(ap)
    else: mark('unsupported parameter type')

def genCall(pr, ap):
    asm.append('call $' + pr.name)

def genRead(x):
    asm.append('call $read')
    y = Var(Int); y.lev = -1

def genWrite(x):
    loadItem(x)
    asm.append('call $write')

def genWriteln():
    asm.append('call $writeln')

def genNewHeap(): # initialized the heap manager
    global heap, memsize
    heap = LinkedList(adr=memsize)
    
def genShowHeap(): # show the information of the heap manager
    print(heap.start)
    
# generate code for explicit allocation
def genNew(x): 
    global memsize
    address = heap.allocate(x,x.adr) # heap manager return the best available adr using first fit
    if type(address) != int: # means reallocate an object that is already allocated
        address = address[0]
        for i in range(0,x.tp.base.size,4):
            asm.append('i32.const '+str(address+i))
            asm.append('i32.const 0')
            asm.append('i32.store')
    asm.append('i32.const '+ str(x.adr)) #return adr (of the object on the heap) to the pointer
    asm.append('i32.const ' + str(address)) # determine the adr that the object start at
    asm.append('i32.store')
    
# generate code for explicit deallocation
def genDispose(x):
    target = heap.search(x.adr)
    if target != None:
        address = target.adr
        for i in range(0,x.tp.base.size,4): #reset value
            asm.append('i32.const '+str(address+i))
            asm.append('i32.const 0')
            asm.append('i32.store')
    if not heap.deallocate(x.adr): # heap manager to release the memory
        mark("error: not an object allocated on the heap!")
    else:
        asm.append('i32.const '+ str(x.adr))
        asm.append('i32.const 20000')
        asm.append('i32.store') # point to a very large address that is not allocated anything
    

def genSeq(x, y):
    pass

def genThen(x):
    loadItem(x)
    asm.append('if')
    return x

def genIfThen(x, y):
    asm.append('end')

def genElse(x, y):
    asm.append('else')

def genIfElse(x, y, z):
    asm.append('end')

def genWhile():
    asm.append('loop')

def genDo(x):
    loadItem(x)
    asm.append('if')
    return x

def genWhileDo(t, x, y):
    asm.append('br 1')
    asm.append('end')
    asm.append('end')
    
